# Predicción de Enfermedad Cardíaca: Explicación Paso a Paso

En este cuaderno vamos a ver cómo una computadora puede ayudarnos a predecir si una persona tiene enfermedad cardíaca (`HeartDisease`). Usaremos dos "modelos" (formas de predecir): **regresión logística** y **Random Forest**. También veremos qué factores de salud son más importantes para la predicción.

## ¿Qué aprenderás aquí?
- **Clasificación binaria**: El objetivo es responder a una pregunta de sí o no: ¿la persona tiene enfermedad cardíaca?
- **Interpretabilidad**: Queremos saber qué factores de salud (como el colesterol, la edad, etc.) influyen más en el resultado.
- **Métricas de clasificación**: Son formas de medir si el modelo acierta o se equivoca, por ejemplo: cuántas veces acierta (accuracy), qué tan bien separa sanos de enfermos (AUC), etc.

> **Nota:** No necesitas saber matemáticas ni programación para entender los conceptos. Los comentarios y explicaciones te guiarán paso a paso.

In [ ]:
# PASO 1: Cargar los datos y ver cómo son
import pandas as pd  # Para manejar tablas de datos
import matplotlib.pyplot as plt  # Para hacer gráficos
import seaborn as sns  # Para hacer gráficos bonitos

# Leemos el archivo CSV con los datos de pacientes
df = pd.read_csv('heart.csv')

# Mostramos las primeras filas para ver cómo son los datos
print(df.head())  # Así vemos qué columnas hay y cómo son los valores

**Exploración:**  
Mira las columnas de la tabla. ¿Cuáles de estos datos crees que pueden estar relacionados con problemas del corazón? Por ejemplo: la edad, el tipo de dolor en el pecho, el sexo, el colesterol, etc.

> **Consejo:** Si no entiendes alguna palabra, ¡pregunta! Lo importante es tener curiosidad.

In [ ]:
# PASO 2: Ver cuántas personas tienen enfermedad cardíaca y cuántas no
# Esto nos ayuda a saber si los datos están equilibrados (hay más sanos o más enfermos)
sns.countplot(x='HeartDisease', data=df)
plt.title('¿Cuántas personas tienen enfermedad cardíaca?')
plt.xlabel('0 = No tiene enfermedad, 1 = Sí tiene enfermedad')
plt.ylabel('Número de personas')
plt.show()

**Pregunta:**  
¿Hay más personas sanas o más personas con enfermedad cardíaca en los datos? Si hay muchos más de un grupo que de otro, el modelo puede aprender peor.

> **Ejemplo:** Si hay 90% sanos y 10% enfermos, el modelo podría aprender a decir siempre "sano" y acertar mucho, pero no sería útil.

In [ ]:
# PASO 3: Ver si el colesterol influye en la enfermedad
# Comparamos los valores de colesterol entre personas sanas y con enfermedad
sns.boxplot(x='HeartDisease', y='Cholesterol', data=df)
plt.title('¿El colesterol es diferente entre sanos y enfermos?')
plt.xlabel('0 = No tiene enfermedad, 1 = Sí tiene enfermedad')
plt.ylabel('Colesterol')
plt.show()

**Análisis:**  
¿Ves que las personas con enfermedad cardíaca suelen tener el colesterol más alto? Si es así, el colesterol podría ser un factor importante.

> **Recuerda:** Un factor importante es aquel que ayuda a diferenciar entre sanos y enfermos.

In [ ]:
# PASO 4: Preparar los datos para el modelo
# Algunas columnas tienen palabras (como tipo de dolor), las convertimos a números (dummies)
cat_vars = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']  # Estas son las columnas con palabras
df = pd.get_dummies(df, columns=cat_vars, drop_first=True)  # Creamos columnas nuevas con 0 y 1

# Mostramos cómo queda la tabla después de convertir a dummies
print(df.head())  # Ahora todo son números, ¡listo para el modelo!

**¿Por qué convertir a dummies?**  
Algunos datos, como el tipo de dolor en el pecho, no son números sino palabras. Los modelos de inteligencia artificial solo entienden números. Por eso, convertimos cada opción en una columna que vale 0 o 1 (por ejemplo: dolor tipo A = 1, si no = 0).

> **Ejemplo:** Si tienes "DolorTipo = A" y "DolorTipo = B", creamos dos columnas: una para A y otra para B, y ponemos 1 o 0 según corresponda.

In [ ]:
# PASO 5: Separar los datos en entrenamiento y prueba
# Así podemos entrenar el modelo con una parte y comprobar si acierta con la otra
from sklearn.model_selection import train_test_split  # Herramienta para separar los datos

# X son las variables que usaremos para predecir, y es la respuesta (enfermedad sí/no)
X = df.drop(columns=['HeartDisease'])  # Todo menos la columna de respuesta
y = df['HeartDisease']  # La columna que queremos predecir

# 70% para entrenar, 30% para probar. random_state=8 hace que siempre salga igual.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

# Ahora tenemos X_train, y_train para entrenar y X_test, y_test para comprobar

In [ ]:
# PASO 6: Crear y probar el modelo de regresión logística
# Este modelo intenta encontrar una fórmula para predecir sí/no
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# Creamos el modelo y lo entrenamos con los datos de entrenamiento
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Hacemos predicciones con los datos de prueba
y_pred = logreg.predict(X_test)

# Mostramos qué porcentaje acierta el modelo (accuracy)
print('Accuracy:', accuracy_score(y_test, y_pred))

# Calculamos el AUC, que mide qué tan bien separa sanos de enfermos
print('AUC:', roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]))

# Mostramos un reporte más detallado (precisión, recall, etc.)
print('Reporte de clasificación:\n', classification_report(y_test, y_pred))

# Mostramos la matriz de confusión: cuántos aciertos y errores
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)
plt.title('Matriz de confusión')
plt.xlabel('Predicción del modelo')
plt.ylabel('Valor real')
plt.show()

**¿Qué es el AUC?**  
El AUC (Area Under Curve) es una forma de medir qué tan bien el modelo distingue entre personas sanas y personas con enfermedad. Si el AUC es 1, el modelo es perfecto; si es 0.5, el modelo no sabe distinguir (es como lanzar una moneda).

> **En resumen:** Cuanto más cerca de 1, mejor.

In [ ]:
# PASO 7: Probar un modelo más avanzado: Random Forest
# Random Forest es un modelo que combina muchos árboles de decisión para mejorar la predicción
from sklearn.ensemble import RandomForestClassifier

# Creamos el modelo y lo entrenamos
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Hacemos predicciones con Random Forest
y_pred_rf = rf.predict(X_test)

# Mostramos el porcentaje de aciertos y el AUC
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred_rf))
print('Random Forest AUC:', roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))

#### Reflexión final para todos:

- ¿Qué modelo acierta más, la regresión logística o el Random Forest? ¿Por qué crees que ocurre esto?
- ¿Qué factores de salud (variables) parecen ser los más importantes según los modelos?
- ¿Crees que sería útil en la vida real poder predecir así este tipo de enfermedades? ¿Por qué?

> **Recuerda:** Lo importante no es solo acertar, sino también entender por qué el modelo toma sus decisiones.